## Importamos las dependencias

In [ ]:
!pip install langchain_core
!pip install langchain
!pip install langchain_openai
!pip install -qU langchain-qdrant
!pip install openai
!pip install

ERROR: You must give at least one requirement to install (see "pip help install")


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor
from langchain_openai.chat_models import ChatOpenAI
from langchain_qdrant import QdrantVectorStore
from langchain.tools import tool
from qdrant_client.http import models
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
import json
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["QDRANT_URL"] = ""
os.environ["QDRANT_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = ""

llm = ChatOpenAI(model_name="gpt-4o-mini")


# Creando los tools

In [ ]:
@tool
def search_JSON(json_input) -> str:
    """
    Return all the festive days in a city.
    Example of input JSON:
    {
        "city": "city you want to get festive days about"
    }
    """
    # Parse input JSON
    parsed_json = json.loads(json_input)
    city = parsed_json.get("city", None)

    if not city:
        return json.dumps({"error": "City not provided."}, ensure_ascii=False)

    # Path to the JSON file containing festive days
    json_location = "/content/festivos_andalucia (1).json"

    with open(json_location, "r", encoding="utf-8") as file:
        festive_data = json.load(file)

    # Fetch festive days for the given city
    festive_days = festive_data.get(city, None)

    if festive_days:
        return json.dumps({"city": city, "festive_days": festive_days}, ensure_ascii=False, indent=4)
    else:
        return json.dumps({"error": f"No data found for city: {city}"}, ensure_ascii=False)

In [ ]:
@tool
def search_vdb(json_input) -> str:
    """
    Retrieves very useful information about different cities in Andalucia

    Example of input JSON:
    {
        "query": "topic you want to get information about",
        "city": "city you want to get information about"
    }

    Example of output:
    {
    }
    """
    parsed_json = json.loads(json_input)

    topic = parsed_json.get("query", None)
    city=parsed_json.get("city", None)



    qdrant_client = QdrantClient(
      url = os.environ["QDRANT_URL"],
      api_key = os.environ["QDRANT_API_KEY"]
    )


    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

    vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name= "mi_primer_rag",
    embedding=embeddings,
    )

    # Perform similarity search on the precision collection
    parsed_json = json.loads(json_input)

    location = parsed_json.get("query", None)

    results = vector_store.similarity_search(
    query=topic,
    k =3,  # Número de resultados a devolver
    )
    retrieved_docs = []
    for document in results:
        # print(document)
        retrieved_docs.append(document.page_content)
    return json.dumps(retrieved_docs)

## Creamos el PromptTemplate del agente ReAct

In [ ]:
prompt = PromptTemplate(
        input_variables=["input", "tool_names", "agent_scratchpad"],
        template="""
        You are an intelligent assistant that always thinks in English using the tools: {tool_names}.
        Here are the available tools: {tools}

        If you need to call a tool, respond using this exact format:

        Thought: [Explain your reasoning]
        Action: Tool to call
        Action Input: Tool input in JSON format with no additional quotes around the entire input

        If a tool response is received:
        - Extract all the relevant information to give a proper answer to user query.
        - Provide a final answer in the following format:

        Final Answer: [Your final response to the query: {input}. Include all relevant information with details.Answer always in spanish]

        Query: {input}

        {agent_scratchpad}
        """
        )

In [ ]:
tools = [search_JSON, search_vdb]

In [ ]:
react_agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor.from_agent_and_tools(
        react_agent, tools=tools, handle_parsing_errors=True
    )

In [ ]:
agent_response = agent_executor.invoke(
                {
                    "input": "¿Cuales son los festivos en Sevilla?"
                }
            )
print(agent_response)

{'input': '¿Cuales son los festivos en Sevilla?', 'output': 'Los días festivos en Sevilla son los siguientes: \n- 1 de enero: Año Nuevo\n- 6 de enero: Epifanía del Señor\n- 28 de febrero: Día de Andalucía\n- 17 de abril: Jueves Santo\n- 18 de abril: Viernes Santo\n- 1 de mayo: Fiesta del Trabajo\n- 15 de agosto: Asunción de la Virgen\n- 13 de octubre: Fiesta Nacional de España (trasladada al lunes)\n- 1 de noviembre: Día de Todos los Santos\n- 6 de diciembre: Día de la Constitución Española\n- 8 de diciembre: Inmaculada Concepción\n- 25 de diciembre: Natividad del Señor\n- 18 de abril: Lunes de Feria\n- 30 de mayo: San Fernando'}
